In [6]:
import sensordata 
import numpy as np
gauss_a,gauss_b,wind_a,wind_b,labels = sensordata.data_generate("mat/gauss_mat/","gauss_feat","mat/wind_mat/","wind_vel_feat")

In [8]:
datadir = "train-npy/"
np.save(datadir + 'gauss_a_set_4', gauss_a)
np.save(datadir + 'gauss_b_set_4', gauss_b)
np.save(datadir + 'wind_a_set_4', wind_a)
np.save(datadir + 'wind_b_set_4', wind_b)
np.save(datadir + 'labels_set_4', labels)

In [15]:
import torch; torch.utils.backcompat.broadcast_warning.enabled = True
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.auto import tqdm

datadir = "train-npy/"
true_gauss = np.load(datadir + 'gauss_a_set_4.npy')
true_wind = np.load(datadir + 'wind_a_set_4.npy')
wrong_gauss = np.load(datadir + 'gauss_b_set_4.npy')
wrong_wind = np.load(datadir + 'wind_b_set_4.npy')
label = np.load(datadir + 'labels_set_4.npy')


class DummyDataset(Dataset):
    """
    This class should contain complete dataset  in init 
    """

    def __init__(self, genuine_geo,genuine_wind,forged_geo,forged_wind,labels):
        super().__init__()
        self.true_gauss = genuine_geo
        self.true_wind = genuine_wind
        self.wrong_gauss = forged_geo
        self.wrong_wind = forged_wind
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        true_geo_tensor = torch.tensor(true_gauss[idx],dtype=torch.float)
        true_wind_tensor = torch.tensor(true_wind[idx],dtype=torch.float) # this is complete dataset

        wrong_geo_tensor = torch.tensor(wrong_gauss[idx],dtype=torch.float)
        wrong_wind_tensor = torch.tensor(wrong_wind[idx],dtype=torch.float)
        #convert_label = np.array(label[idx])
        labels = torch.tensor(self.labels[idx],dtype=torch.float)
        return true_wind_tensor,true_geo_tensor,wrong_wind_tensor,wrong_geo_tensor,labels


In [20]:
traindataset = DummyDataset(true_gauss[0:6],true_wind[0:6],wrong_gauss[0:6],wrong_wind[0:6],label[0:6])
train_dataloader = DataLoader(traindataset, batch_size=128, shuffle=True)
type(tqdm(enumerate(train_dataloader),total=len(train_dataloader)))
#for steps, (true_wind_tensor, true_geo_tensor, wrong_wind_tensor, wrong_geo_tensor, labels) in tqdm(enumerate(train_dataloader),total=len(train_dataloader)):
        #print(true_wind_tensor.shape)
        #break

  0%|          | 0/1 [00:00<?, ?it/s]

tqdm.auto.tqdm